In [26]:
import pandas as pd
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import torch
from torch import nn
import torch.optim as optimzer
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import TensorDataset, DataLoader

CATEGORIES = ["buildings", "forest",
              "glacier", "mountain", 
              "sea", "street"]
EPOCHS = 50
TEST_SIZE = 0.30

In [27]:
def load_Intel_Image(dataDirectory : str):
    images = []
    labels = []
    for SingleCategory in CATEGORIES:
        category = os.path.join(dataDirectory, str(SingleCategory))
        
        for image in os.listdir(category):
            img_path = os.path.join(category, image)
            final_image = cv2.imread(img_path)
            
            if final_image is None:
                raise Exception("Problem with the images")
            img = cv2.resize(final_image, (64,64)) #Making the Picture 64 x 64
            images.append(img)
            labels.append(SingleCategory)
    
    return (np.array(images), np.array(labels))

            
        
            

In [28]:
images, labels = load_Intel_Image("seg_train/")

X_train, X_test, Y_train, Y_test = train_test_split(
    np.array(images), np.array(labels), test_size=TEST_SIZE, random_state=42
)

print(f"This is the shape of X_train: {X_train.shape}")
print(f"This is the shape of Y_train: {Y_train.shape}")
print(f"This is the shape of X_test: {X_test.shape}")
print(f"This is the shape of Y_Test: {Y_test.shape}")

This is the shape of X_train: (9823, 64, 64, 3)
This is the shape of Y_train: (9823,)
This is the shape of X_test: (4211, 64, 64, 3)
This is the shape of Y_Test: (4211,)


In [29]:
#1) Normalize Input Pixels
X_train = X_train.astype("float32") / 255.0
X_test = X_test.astype("float32") / 255.0


#2) We need to Transpose the Matrix because TF (M, H, W, NC) -> Pytorch (M, NC, H, W)
X_train = np.transpose(X_train, (0,3,1,2))
X_test = np.transpose(X_test, (0,3,1,2))

#3) Codificate the Y to Labels Encoder
Label_Encoder = LabelEncoder()
Y_train_encoded = Label_Encoder.fit_transform(Y_train)
Y_test_encoded = Label_Encoder.fit_transform(Y_test)

#4) Convert tensor of Pytorch(The numpy array of TensorFlow)
x_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(Y_train_encoded, dtype=torch.long)
x_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(Y_test_encoded, dtype=torch.long)

In [30]:
train_Dataset = TensorDataset(x_train, y_train)
test_DataSet = TensorDataset(x_test, y_test)

train_loader = DataLoader(train_Dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_DataSet, batch_size=64, shuffle=True)


In [31]:
class CNN_Model(nn.Module):
    def __init__(self, num_classes=6):
        super(CNN_Model, self).__init__()
        
        #First Block
        self.zero_pad = nn.ZeroPad2d(3) #Padding (3,3s)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, stride=2)
        self.bn1 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(kernel_size=3, stride=2)
        
        #Second Block
        self.covn2 = nn.Conv2d(in_channels=64, out_channels=32, kernel_size=4, stride=2)
        self.bn2 = nn.BatchNorm2d(32)
        self.pool2 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        #Third Block
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=2)
        self.bn3 = nn.BatchNorm2d(32)
        self.pool3 = nn.MaxPool2d(kernel_size=3, stride=1,padding=1)
        
        self.fc = nn.Linear(32 * 2 * 2, num_classes)
    
    
    def forward(self, x):
        #Block 1
        x = self.zero_pad(x)
        x = nn.functional.relu(self.bn1(self.conv1(x)))
        x = self.pool1(x)
        
        #Block 2
        x = nn.functional.relu(self.bn2(self.covn2(x)))
        x = self.pool2(x)
        
        #Block 3 
        x = nn.functional.relu(self.bn3(self.conv3(x)))
        x = self.pool3(x)
        
        #Flatten and Final Block
        x = torch.flatten(x, 1)
        x = self.fc(x)
        #Dim = 1 == np.sum() == 1
        x = nn.functional.softmax(x, dim=1)

        return x
        
        
        

In [34]:
convModel = CNN_Model(num_classes=6)
criterion = nn.CrossEntropyLoss()
optim = optimzer.AdamW(convModel.parameters(), lr=5e-5)

In [35]:
for epoch in range(EPOCHS):
    convModel.train()
    epoch_loss = 0.0
    
    for batch_x, batch_y in train_loader:
        optim.zero_grad()
        outputs = convModel(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optim.step()
        epoch_loss += loss.item()
        
    print(f"Epoch [{epoch+1}/{EPOCHS}], Loss: {epoch_loss/len(train_loader):.4f}")

Epoch [1/50], Loss: 1.6972
Epoch [2/50], Loss: 1.6039
Epoch [3/50], Loss: 1.5491
Epoch [4/50], Loss: 1.5010
Epoch [5/50], Loss: 1.4595
Epoch [6/50], Loss: 1.4279
Epoch [7/50], Loss: 1.3995
Epoch [8/50], Loss: 1.3771
Epoch [9/50], Loss: 1.3561
Epoch [10/50], Loss: 1.3396
Epoch [11/50], Loss: 1.3245
Epoch [12/50], Loss: 1.3107
Epoch [13/50], Loss: 1.3009
Epoch [14/50], Loss: 1.2907
Epoch [15/50], Loss: 1.2818
Epoch [16/50], Loss: 1.2728
Epoch [17/50], Loss: 1.2615
Epoch [18/50], Loss: 1.2536
Epoch [19/50], Loss: 1.2459
Epoch [20/50], Loss: 1.2401
Epoch [21/50], Loss: 1.2332
Epoch [22/50], Loss: 1.2267
Epoch [23/50], Loss: 1.2224
Epoch [24/50], Loss: 1.2173
Epoch [25/50], Loss: 1.2103
Epoch [26/50], Loss: 1.2052
Epoch [27/50], Loss: 1.2000
Epoch [28/50], Loss: 1.1951
Epoch [29/50], Loss: 1.1904
Epoch [30/50], Loss: 1.1865
Epoch [31/50], Loss: 1.1832
Epoch [32/50], Loss: 1.1778
Epoch [33/50], Loss: 1.1744
Epoch [34/50], Loss: 1.1711
Epoch [35/50], Loss: 1.1683
Epoch [36/50], Loss: 1.1645
E

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
import torch


convModel.eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for batch_x, batch_y in test_loader:
        outputs = convModel(batch_x)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(batch_y.cpu().numpy())


acc = accuracy_score(all_labels, all_preds)
prec = precision_score(all_labels, all_preds, average='macro')
rec = recall_score(all_labels, all_preds, average='macro')
f1 = f1_score(all_labels, all_preds, average='macro')


print(f"✅ Validation Metrics:")
print(f"Accuracy:  {acc*100:.2f}%")
print(f"Precision: {prec*100:.2f}%")
print(f"Recall:    {rec*100:.2f}%")
print(f"F1-score:  {f1*100:.2f}%")


✅ Validation Metrics:
Accuracy:  72.60%
Precision: 74.97%
Recall:    72.56%
F1-score:  72.27%


In [ ]:
def evaluate_model(model, loader, criterion, name="Validation"):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_x, batch_y in loader:
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            total_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            total += batch_y.size(0)
            correct += (preds == batch_y).sum().item()

    avg_loss = total_loss / len(loader)
    acc = correct / total
    print(f"Accuracy of the {name}: {acc:.4f}")
    print(f"Loss of the {name}: {avg_loss:.4f}")
    return acc, avg_loss

# Evaluate Model
train_acc, train_loss = evaluate_model(convModel, train_loader, criterion, "training")
test_acc, test_loss = evaluate_model(convModel, test_loader, criterion, "testing")


Accuracy of the training: 0.9092
Loss of the training: 1.1526
Accuracy of the testing: 0.7260
Loss of the testing: 1.3240
